# Create the Channel Means Feature set

In [30]:
# import toolboxes
import pandas as pd
import numpy as np
import os
from collections import Counter

In [31]:
directory = "source_data/"
filename = "EEG_data.csv"

df = pd.read_csv("source_data/EEG_data.csv")

In [32]:
df.shape

(12811, 15)

In [33]:
df.isnull().sum()

SubjectID             0
VideoID               0
Attention             0
Mediation             0
Raw                   0
Delta                 0
Theta                 0
Alpha1                0
Alpha2                0
Beta1                 0
Beta2                 0
Gamma1                0
Gamma2                0
predefinedlabel       0
user-definedlabeln    0
dtype: int64

In [34]:
print("Below are subject IDs and their counts")
Counter(df["user-definedlabeln"])

Below are subject IDs and their counts


Counter({0.0: 6244, 1.0: 6567})

In [35]:
print("Below are subject IDs and their counts")
Counter(df["VideoID"])

Below are subject IDs and their counts


Counter({0.0: 1412,
         1.0: 1414,
         2.0: 1274,
         3.0: 1206,
         4.0: 1356,
         5.0: 1230,
         6.0: 1181,
         7.0: 1177,
         8.0: 1280,
         9.0: 1281})

In [17]:
# sample rate of 2 Hz (1 sample every 0.5 seconds)
# so it's basically time series data
# data's kaggle site says theta seems to be where confusion is held
# this is so stupid they just used the means of each channel as their features
# need to seperate the combos of videos and participants

In [39]:
# create an empty numpy array
eeg_arr = np.zeros((100, 14))
eeg_arr.shape

(100, 14)

In [40]:
eeg_arr

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [41]:
counter = 0
for i in range(10):
    for j in range(10):
        curr = df.loc[
            (df["user-definedlabeln"] == i) & (df["VideoID"] == j)
        ].copy()  # SubjectID, VideoID
        curr_attention = np.mean([curr["Attention"]])
        curr_mediation = np.mean([curr["Mediation"]])
        curr_raw = np.mean([curr["Raw"]])
        curr_delta = np.mean([curr["Delta"]])
        curr_theta = np.mean([curr["Theta"]])
        curr_alpha1 = np.mean([curr["Alpha1"]])
        curr_alpha2 = np.mean([curr["Alpha2"]])
        curr_beta1 = np.mean([curr["Beta1"]])
        curr_beta2 = np.mean([curr["Beta2"]])
        curr_gamma1 = np.mean([curr["Gamma1"]])
        curr_gamma2 = np.mean([curr["Gamma2"]])
        label = np.mean(curr["user-definedlabeln"])

        eeg_arr[counter, :] = [
            i,
            j,
            label,
            curr_attention,
            curr_mediation,
            curr_raw,
            curr_delta,
            curr_theta,
            curr_alpha1,
            curr_alpha2,
            curr_beta1,
            curr_beta2,
            curr_gamma1,
            curr_gamma2,
        ]
        # Attention Mediation Raw Delta Theta Alpha1 Alpha2 Beta1 Beta2 Gamma1 Gamma2
        counter += 1

/home/codespace/.local/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/codespace/.local/lib/python3.10/site-packages/numpy/core/_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [46]:
df_new = pd.DataFrame(
    eeg_arr,
    columns=[
        "SubjectID",
        "VideoID",
        "user-definedlabel",
        "Attention",
        "Mediation",
        "Raw",
        "Delta",
        "Theta",
        "Alpha1",
        "Alpha2",
        "Beta1",
        "Beta2",
        "Gamma1",
        "Gamma2",
    ],
)

In [47]:
df_new.head()

,SubjectID,VideoID,user-definedlabel,Attention,Mediation,Raw,Delta,Theta,Alpha1,Alpha2,Beta1,Beta2,Gamma1,Gamma2
0,0.0,0.0,0.0,50.940283,52.597166,37.977733,451202.998988,131856.127530,31017.000000,24903.787449,17933.574899,21402.362348,16434.486842,7064.616397
1,0.0,1.0,0.0,42.586572,40.183746,89.590106,635639.577739,161059.061837,38064.955830,33443.510601,24692.674912,95066.833922,80776.837456,23234.597173
2,0.0,2.0,0.0,45.012064,45.718499,69.626005,503787.986595,129700.764075,29806.321716,25791.596515,19972.315013,61132.560322,49830.329759,18302.111260
3,0.0,3.0,0.0,34.901606,40.014056,36.435743,505448.068273,122171.409639,29916.524096,19385.267068,14105.273092,30998.883534,17503.224900,6903.518072
4,0.0,4.0,0.0,50.960106,54.167553,38.390957,509832.273936,139927.718085,34568.563830,26043.888298,17149.888298,22354.481383,19335.125000,9019.875000


In [48]:
# save to a CSV so you don't have to keep running this each time
df_new.to_csv("cleaned_data/meanfeatures.csv")

## Standardize variables

In [49]:
# import sklearn standard scaler
from sklearn.preprocessing import StandardScaler

# create a scaler object
scaler = StandardScaler()

# fit and transform the data
target = df_new["user-definedlabel"]
features = df_new.drop(["user-definedlabel"], axis=1)

scaled_features = scaler.fit_transform(features)

In [51]:
features = pd.DataFrame(
    scaled_features,
    columns=[
        "SubjectID",
        "VideoID",
        "Attention",
        "Mediation",
        "Raw",
        "Delta",
        "Theta",
        "Alpha1",
        "Alpha2",
        "Beta1",
        "Beta2",
        "Gamma1",
        "Gamma2",
    ],
)

In [52]:
# add back target to features
features["user-definedlabel"] = target

In [53]:
# export
features.to_csv("cleaned_data/meanfeatures_scaled.csv")

Bad pipe message: %s [b'k\xed4\x88A\xab\xd3\xef\x8aZ\x8f\nI\xaa\xb9a$I\x00\x00|\xc0,\xc00\x00\xa3\x00\x9f\xcc\xa9\xcc\xa8\xcc\xaa\xc0\xaf\xc0\xad\xc0\xa3\xc0\x9f\xc0]', b"\xc0W\xc0S\xc0+\xc0/\x00\xa2\x00\x9e\xc0\xae\xc0\xac\xc0\xa2\xc0\x9e\xc0\\\xc0`\xc0V\xc0R\xc0$\xc0(\x00k\x00j\xc0#\xc0'\x00g\x00@\xc0\n\xc0\x14\x009\x008\xc0\t\xc0\x13\x003\x002\x00\x9d\xc0\xa1\xc0\x9d\xc0Q\x00\x9c\xc0\xa0\xc0\x9c\xc0P\x00=\x00<\x005\x00/\x00\x9a\x00\x99\xc0\x07\xc0\x11\x00\x96\x00\x05\x00\xff\x01\x00\x00j\x00\x00\x00\x0e\x00\x0c\x00\x00\t127.0.0.1\x00\x0b\x00\x04\x03\x00\x01\x02\x00\n\x00\x0c\x00\n\x00\x1d\x00\x17\x00\x1e\x00\x19\x00\x18\x00#\x00\x00\x00\x16\x00\x00\x00\x17\x00\x00\x00\r\x000\x00.\x04\x03\x05\x03\x06\x03\x08\x07\x08\x08\x08\t\x08\n\x08\x0b\x08\x04\x08\x05\x08\x06\x04\x01\x05\x01\x06\x01\x03", b'\x03\x03']
Bad pipe message: %s [b'']
Bad pipe message: %s [b'', b'\x02']
Bad pipe message: %s [b'\x05\x02\x06']
Bad pipe message: %s [b'\xd3\xca&\x8fp\x14\x8c\x0bK/\xc8\xc6U\xc9p\xcd\xaa\xeb\